## Env setup

In [1]:
# ==== USER CONFIG (edit these) ====
NEURVPS_ROOT = "/users/project1/pt01183/Building-height-width/external/neurvps"  # repo root that contains the 'neurvps/' package
IN_DIR       = "/users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq"
OUT_DIR      = "/users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/my5_vpts"
CFG_YAML     = "/users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/config_gsv.yaml"
CKPT_PATH    = "/users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/checkpoint_state_anet.pth"

# Use "cuda:0" if you're inside a GPU session; otherwise "cpu"
DEVICE       = "cuda:0"


In [2]:
import sys, os, pathlib

def ensure_neurvps_on_path(repo_root: str):
    repo_root = os.path.abspath(repo_root)
    candidates = [repo_root, os.path.join(repo_root, "src")]
    added = False
    for c in candidates:
        if os.path.isdir(os.path.join(c, "neurvps")):
            if c not in sys.path:
                sys.path.insert(0, c)
                added = True
                break
    if not added:
        raise RuntimeError(
            f"Could not find 'neurvps' package under {repo_root} (or {repo_root}/src). "
            "Check NEURVPS_ROOT and repo layout."
        )

ensure_neurvps_on_path(NEURVPS_ROOT)

# quick sanity print
import importlib
neurvps = importlib.import_module("neurvps")
print("neurvps package path ->", getattr(neurvps, "__file__", "(unknown)"))


neurvps package path -> /users/project1/pt01183/Building-height-width/external/neurvps/neurvps/__init__.py


In [3]:
import shutil, pathlib
shutil.rmtree(pathlib.Path.home()/".cache"/"torch_extensions", ignore_errors=True)


In [4]:
from pathlib import Path
import re

NEUR = Path("/users/project1/pt01183/Building-height-width/external/neurvps")

targets = [
    NEUR/"neurvps/models/cpp/deform_conv.cpp",
    NEUR/"neurvps/models/cpp/deform_conv_cuda.cu",
]

def patch_text(s: str) -> str:
    t = s
    # Deprecated .type() API → modern API
    t = t.replace(".type().is_cuda()", ".is_cuda()")
    t = t.replace(".type().scalarType()", ".scalar_type()")
    # AT_DISPATCH...(... .type(), ...) → (... .scalar_type(), ...)
    t = re.sub(r"AT_DISPATCH_FLOATING_TYPES_AND_HALF\(\s*([A-Za-z0-9_]+)\.type\(\)",
               r"AT_DISPATCH_FLOATING_TYPES_AND_HALF(\1.scalar_type()", t)
    t = re.sub(r"AT_DISPATCH_FLOATING_TYPES\(\s*([A-Za-z0-9_]+)\.type\(\)",
               r"AT_DISPATCH_FLOATING_TYPES(\1.scalar_type()", t)
    return t

for p in targets:
    src = p.read_text()
    new = patch_text(src)
    if new != src:
        p.write_text(new)
        print("Patched:", p)
    else:
        print("No changes needed:", p)


No changes needed: /users/project1/pt01183/Building-height-width/external/neurvps/neurvps/models/cpp/deform_conv.cpp
No changes needed: /users/project1/pt01183/Building-height-width/external/neurvps/neurvps/models/cpp/deform_conv_cuda.cu


In [58]:
# GPU + Torch sanity
import sys, subprocess, os
print("Python:", sys.version)

# show GPU/driver
try:
    out = subprocess.check_output(["nvidia-smi"], text=True)
    print(out.splitlines()[0])
except Exception as e:
    print("nvidia-smi not available:", e)

# torch check
try:
    import torch
    print("Torch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA runtime (torch):", torch.version.cuda)
        print("GPU:", torch.cuda.get_device_name(0))
        x = torch.rand(2,2).cuda()
        print("Tensor device:", x.device)
except Exception as e:
    print("Torch not installed or failed to import:", e)


Python: 3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]
Mon Oct 13 16:53:37 2025       
Torch: 2.6.0+cu124
CUDA available: True
CUDA runtime (torch): 12.4
GPU: NVIDIA A100 80GB PCIe
Tensor device: cuda:0


## Make a GSV-specific config (sets the correct focal)

Use this first. It tells NeurVPS the focal that matches your 640×640, FOV = 100° images.

In [56]:
from pathlib import Path
import yaml, math

REPO   = Path("/users/project1/pt01183/Building-height-width")
NEUR   = REPO / "external" / "neurvps"
BASE   = NEUR / "logs" / "tmm17" / "config.yaml"        # downloaded from HF
CFG_GSV= NEUR / "logs" / "tmm17" / "config_gsv.yaml"

# your GSV camera numbers (from earlier): fov=100°, width=640
fov_deg = 100.0
W = 640
f_px = (W/2.0) / math.tan(math.radians(fov_deg/2.0))   # ≈ 268.51 px

with open(BASE, "r") as f:
    cfg = yaml.safe_load(f)

# Ensure expected structure exists
cfg.setdefault("data", {})
cfg["data"]["focal"] = float(f_px)

# IMPORTANT: the TMM17 checkpoint expects NO conic/DCN branch:
cfg.setdefault("model", {})
cfg["model"]["conic_6x"] = False   # <- critical
# (leave the rest of model settings as-is; do not add new keys)

with open(CFG_GSV, "w") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)

print("Wrote:", CFG_GSV)
print("focal(px):", f_px)




Wrote: /users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/config_gsv.yaml
focal(px): 268.51188197672957


## Gather your 5 RGBs (already 640×640) into a small input folder

In [3]:
from pathlib import Path
import shutil, glob

REPO = Path("/users/project1/pt01183/Building-height-width")
NEURVPS = REPO / "external" / "neurvps"

# <-- your actual RGB source folder
RAW_DIR = REPO / "Gdańsk, Poland" / "save_rgb" / "imgs"

KEEP_IDS = ["6_196","2_190","7_4","8_139","9_196"]

# We'll just copy to a small working folder for NeurVPS (no resize needed)
SQ_DIR  = NEURVPS / "data" / "my5_sq"
SQ_DIR.mkdir(parents=True, exist_ok=True)

def copy_by_id(src_dir: Path, sid: str, dst_dir: Path) -> Path | None:
    # try common extensions; if filenames have suffixes, also try glob with sid.*
    candidates = []
    for ext in (".jpg",".jpeg",".png",".JPG",".PNG"):
        p = src_dir / f"{sid}{ext}"
        if p.exists():
            candidates.append(p)
    if not candidates:
        # fallback: any file that starts with id (e.g., sid_*.jpg)
        candidates = [Path(p) for p in glob.glob(str(src_dir / f"{sid}.*"))]
    if not candidates:
        return None
    src = sorted(candidates, key=lambda x: x.suffix.lower())[0]
    dst = dst_dir / (sid + src.suffix.lower())
    shutil.copy2(src, dst)
    return dst

copied = []
for sid in KEEP_IDS:
    outp = copy_by_id(RAW_DIR, sid, SQ_DIR)
    if outp:
        copied.append(outp.name)
    else:
        print(f"[WARN] Could not find an image for ID {sid} in {RAW_DIR}")

print(f"Copied {len(copied)} images to {SQ_DIR}:")
for name in copied:
    print(" -", name)

Copied 5 images to /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq:
 - 6_196.jpg
 - 2_190.jpg
 - 7_4.jpg
 - 8_139.jpg
 - 9_196.jpg


## Create the runner script (one-time)

This is a small helper we write ourselves so you can run a folder and get per-image JSONs.

In [16]:
# --- CONFIG ---
REPO_ROOT      = "/users/project1/pt01183/Building-height-width"
NEURVPS_ROOT   = f"{REPO_ROOT}/external/neurvps"

IN_DIR         = f"{NEURVPS_ROOT}/data/my5_sq"
OUT_DIR        = f"{NEURVPS_ROOT}/logs/tmm17/my5_vpts"
CFG_YAML       = f"{NEURVPS_ROOT}/logs/tmm17/config_gsv.yaml"
CKPT_PATH      = f"{NEURVPS_ROOT}/logs/tmm17/checkpoint_state_anet.pth"  # or *_anetfix.pth

import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


Device: cuda:0


In [20]:
import importlib, pkgutil, inspect, types, sys
import torch, torch.nn as nn

def _wrap_backbone_output(bb: nn.Module) -> nn.Module:
    """Ensure backbone(x) returns a tuple/list so [...][0] works in VanishingNet."""
    class _Wrap(nn.Module):
        def __init__(self, core): 
            super().__init__(); self.core = core
        def forward(self, x):
            y = self.core(x)
            if isinstance(y, (list, tuple)): 
                return y
            return (y,)  # make it indexable
    return _Wrap(bb)

def _instantiate_candidate(obj, M):
    try:
        # try with config first
        return obj(M)
    except Exception:
        try:
            return obj()
        except Exception:
            return None

def autodiscover_backbone(M):
    """
    Search neurvps.models.* for anything that looks like a backbone builder
    or an nn.Module class we can instantiate.
    """
    discovered = []

    # 1) Known builder modules first
    for modname in ("neurvps.models.builder", "neurvps.models.backbone", "neurvps.models.backbones"):
        try:
            m = importlib.import_module(modname)
        except Exception:
            continue
        for name in dir(m):
            obj = getattr(m, name, None)
            # candidate factory functions
            if callable(obj) and any(key in name.lower() for key in ["build", "make", "backbone", "resnet"]):
                try:
                    bb = _instantiate_candidate(obj, M)
                    if isinstance(bb, nn.Module):
                        return _wrap_backbone_output(bb)
                except Exception:
                    pass

    # 2) Broad crawl under neurvps.models
    try:
        base = importlib.import_module("neurvps.models")
    except Exception as e:
        raise RuntimeError(f"Cannot import neurvps.models: {e}")
    prefix = base.__name__ + "."
    for finder, name, ispkg in pkgutil.walk_packages(base.__path__, prefix):
        try:
            mod = importlib.import_module(name)
        except Exception:
            continue
        for _, cls in inspect.getmembers(mod, inspect.isclass):
            if not issubclass(cls, nn.Module):
                continue
            # Heuristics: likely backbone names
            if any(key in cls.__name__.lower() for key in ["backbone", "resnet", "vgg", "hrnet", "swin"]):
                bb = _instantiate_candidate(cls, M)
                if isinstance(bb, nn.Module):
                    return _wrap_backbone_output(bb)

    return None  # not found


In [21]:
import torch.nn as nn
from torchvision import models

def make_torchvision_backbone(which="resnet50", pretrained=False, out_layer="layer3"):
    """
    Returns a backbone nn.Module that outputs a feature map (and wraps to tuple).
    out_layer: "layer3" (stride 16) is a good default for dense heads.
    """
    if which == "resnet50":
        m = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)
        C = 1024 if out_layer == "layer3" else 2048
    elif which == "resnet34":
        m = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1 if pretrained else None)
        C = 256 if out_layer == "layer3" else 512
    else:
        raise ValueError("Unsupported backbone name")

    # Build a feature extractor up to desired layer
    layers = [
        ("conv1", m.conv1),
        ("bn1",   m.bn1),
        ("relu",  m.relu),
        ("maxpool", m.maxpool),
        ("layer1", m.layer1),
        ("layer2", m.layer2),
        ("layer3", m.layer3),
    ]
    if out_layer == "layer4":
        layers.append(("layer4", m.layer4))

    feat = nn.Sequential(*(getattr(nn, 'Identity')() if False else layer for name, layer in layers))
    # Wrap so forward returns (feat,)
    class FeatWrap(nn.Module):
        def __init__(self, core): super().__init__(); self.core = core
        def forward(self, x): return (self.core(x),)
    return FeatWrap(feat)


In [ ]:
import yaml, re, types, importlib
import torch, torch.nn as nn

def _extract_state(raw):
    if isinstance(raw, dict):
        for k in ("state_dict","model_state_dict"):
            if k in raw and isinstance(raw[k], dict):
                return raw[k]
    return raw

def _strip_wrappers(k: str) -> str:
    for pref in ("module.", "model."):
        if k.startswith(pref): return k[len(pref):]
    return k

def _remap_state_for_model(state, model_keys):
    has_anet_in_model = any(k.startswith("anet.") for k in model_keys)
    has_top_backbone  = any(k.startswith("backbone.") for k in model_keys)
    has_anet_backbone = any(k.startswith("anet.backbone.") for k in model_keys)
    out = {}
    for k, v in state.items():
        kk = _strip_wrappers(k)
        if has_top_backbone and not has_anet_backbone and kk.startswith("anet.backbone."):
            kk = kk[len("anet."):]
        if has_anet_in_model and not kk.startswith("anet."):
            if kk.startswith("backbone.") or re.match(r"^(fc[0-3]|bn[1-4]|conv[1-4]|score)(\.|$)", kk):
                kk = "anet." + kk
        if not has_anet_in_model and kk.startswith("anet."):
            kk = kk[len("anet."):]
        out[kk] = v
    return out

def load_model_strict_yaml(cfg_path: str, ckpt_path: str, device: str):
    # Imports
    cfgmod = importlib.import_module("neurvps.config")
    vn_mod = importlib.import_module("neurvps.models.vanishing_net")
    VanishingNet = vn_mod.VanishingNet

    # YAML
    with open(cfg_path, "r") as f:
        cfg_yaml = yaml.safe_load(f) or {}
    data_cfg  = (cfg_yaml.get("data")  or {})
    model_cfg = (cfg_yaml.get("model") or {})

    # Propagate scalars + lists to M
    for k, v in model_cfg.items():
        if isinstance(v, (int,float,bool,str)) or k in {"multires","resolutions"}:
            try: setattr(cfgmod.M, k, v)
            except Exception: pass

    # Ensure essentials
    defaults = {
        "fc_channel": 1024,
        "multires": [0.01,0.05,0.2,0.8],
        "smp_pos": 1, "smp_neg": 1, "smp_rnd": 8, "smp_multiplier": 2.0
    }
    for k, v in defaults.items():
        if not hasattr(cfgmod.M, k): setattr(cfgmod.M, k, v)

    # Constants shim if needed (C.io.num_vpts)
    try:
        consts = importlib.import_module("neurvps.constants")
        if hasattr(consts, "C"):
            C = consts.C
        elif hasattr(consts, "Constants"):
            C = consts.Constants()
        else:
            raise ImportError
    except Exception:
        class _IO: pass
        class _C: pass
        C = _C(); C.io = _IO(); C.io.num_vpts = 3
        import sys as _sys
        _sys.modules["neurvps.constants"] = types.SimpleNamespace(C=C)

    focal = float(data_cfg.get("focal", 1.0))

    # 1) Try to auto-discover a backbone from the repo
    bb = autodiscover_backbone(cfgmod.M)

    # 2) If still None, fallback to torchvision ResNet (pretrained=False to match your env)
    if bb is None:
        print("[info] No repo backbone found; using torchvision resnet50 (layer3).")
        bb = make_torchvision_backbone(which="resnet50", pretrained=False, out_layer="layer3")

    # Build model
    output_stride  = getattr(cfgmod.M, "output_stride", 32)
    upsample_scale = getattr(cfgmod.M, "upsample_scale", 4)
    model = VanishingNet(bb, output_stride, upsample_scale).to(device).eval()

    # Load checkpoint
    raw = torch.load(ckpt_path, map_location=device)
    state = _extract_state(raw)
    mk = list(model.state_dict().keys())
    state = _remap_state_for_model(state, mk)
    state_pruned = {k: v for k, v in state.items() if k in mk}
    incompatible = model.load_state_dict(state_pruned, strict=False)
    print(f"[load] missing={len(getattr(incompatible,'missing_keys',[]))} "
          f"unexpected={len(getattr(incompatible,'unexpected_keys',[]))}")

    # Sanity
    assert isinstance(model.backbone, nn.Module), "backbone is not nn.Module"
    return model, focal


In [22]:
model, focal = load_model_strict_yaml(CFG_YAML, CKPT_PATH, DEVICE)
print("Ready. focal =", focal, "| device =", DEVICE)

RuntimeError: Could not build a backbone nn.Module from the repo.

In [19]:
model, focal = load_model_strict_yaml(CFG_YAML, CKPT_PATH, DEVICE)
print("Ready. focal =", focal, "| device =", DEVICE)

RuntimeError: Could not build a backbone nn.Module from the repo.

In [15]:
run_folder(model, focal, device, IN_DIR, OUT_DIR)
print("Done. JSONs:", OUT_DIR)

Found 5 files in /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq


100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 59.53it/s]

[error] failed on /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq/2_190.jpg: 'Box' object is not callable
[error] failed on /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq/6_196.jpg: 'Box' object is not callable
[error] failed on /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq/7_4.jpg: 'Box' object is not callable
[error] failed on /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq/8_139.jpg: 'Box' object is not callable
[error] failed on /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq/9_196.jpg: 'Box' object is not callable
Done. JSONs: /users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/my5_vpts


In [13]:
import inspect, neurvps
from neurvps.models import vanishing_net as vn

print("VanishingNet.forward signature:", inspect.signature(vn.VanishingNet.forward))
print("\n--- forward source (first ~80 lines) ---")
src = inspect.getsource(vn.VanishingNet.forward)
print("\n".join(src.splitlines()[:80]))


VanishingNet.forward signature: (self, input_dict)

--- forward source (first ~80 lines) ---
    def forward(self, input_dict):
        x = self.backbone(input_dict["image"])[0]
        N, _, H, W = x.shape
        test = input_dict.get("test", False)
        if test:
            c = len(input_dict["vpts"])
        else:
            c = M.smp_rnd + C.io.num_vpts * len(M.multires) * (M.smp_pos + M.smp_neg)
        x = x[:, None].repeat(1, c, 1, 1, 1).reshape(N * c, _, H, W)

        if test:
            vpts = [to_pixel(v) for v in input_dict["vpts"]]
            vpts = torch.tensor(vpts, device=x.device)
            return self.anet(x, vpts).sigmoid()

        vpts_gt = input_dict["vpts"].cpu().numpy()
        vpts, y = [], []
        for n in range(N):

            def add_sample(p):
                vpts.append(to_pixel(p))
                y.append(to_label(p, vpts_gt[n]))

            for vgt in vpts_gt[n]:
                for st, ed in zip([0] + M.multires[:-1], M.multires):
       

## Run NeurVPS on the folder and get JSON VPs

Convert the checkpoint to a pure state_dict

In [63]:
import torch, re
from pathlib import Path

SRC = Path("/users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/checkpoint_state_vps.pth")
DST = SRC.with_name("checkpoint_state_anetfix.pth")

sd = torch.load(SRC, map_location="cpu")
new_sd = {}

for k, v in sd.items():
    nk = k
    # If backbone.* -> anet.backbone.*
    if nk.startswith("backbone."):
        nk = "anet." + nk
    # If it's one of the A-Net head blocks -> add anet.* in front
    elif re.match(r"^(fc[0-3]|bn[1-4]|conv[1-4])(\.|$)", nk):
        nk = "anet." + nk
    # else: leave as-is
    new_sd[nk] = v

print("Examples (old -> new):")
for i, (ok, ov) in enumerate(sd.items()):
    if i >= 5: break
    nk = ok
    if nk.startswith("backbone."):
        nk = "anet." + nk
    elif re.match(r"^(fc[0-3]|bn[1-4]|conv[1-4])(\.|$)", nk):
        nk = "anet." + nk
    print(f"  {ok}  ->  {nk}")

torch.save(new_sd, DST)
print("Wrote:", DST, "num_keys:", len(new_sd))


Examples (old -> new):
  backbone.conv1.weight  ->  anet.backbone.conv1.weight
  backbone.conv1.bias  ->  anet.backbone.conv1.bias
  backbone.bn1.weight  ->  anet.backbone.bn1.weight
  backbone.bn1.bias  ->  anet.backbone.bn1.bias
  backbone.bn1.running_mean  ->  anet.backbone.bn1.running_mean
Wrote: /users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/checkpoint_state_anetfix.pth num_keys: 413


In [65]:
import torch
sd = torch.load("/users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/checkpoint_state_anetfix.pth", map_location="cpu")
print("has anet?:", any(k.startswith("anet.") for k in sd.keys()))


has anet?: True


In [78]:
import os, subprocess

REPO = "/users/project1/pt01183/Building-height-width"
NEUR = f"{REPO}/external/neurvps"

in_dir  = f"{NEUR}/data/my5_sq"
out_dir = f"{NEUR}/logs/tmm17/my5_vpts"
cfg_gsv = f"{NEUR}/logs/tmm17/config_gsv.yaml"
ckpt = f"{NEUR}/logs/tmm17/checkpoint_state_anetfix.pth" # <-- use cleaned file

os.makedirs(out_dir, exist_ok=True)

env = os.environ.copy()
env["PYTHONPATH"] = NEUR
env.pop("TORCH_CUDA_ARCH_LIST", None)

cmd = [
    "/mnt/host_scratch/envs/new_geospatial_env/bin/python",
    f"{NEUR}/misc/run_on_folder.py",
    in_dir, out_dir, cfg_gsv, ckpt, "cuda:0"
]
print("Running:\n ", " ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True, env=env)
print(res.stdout)
print(res.stderr)




Running:
  /mnt/host_scratch/envs/new_geospatial_env/bin/python /users/project1/pt01183/Building-height-width/external/neurvps/misc/run_on_folder.py /users/project1/pt01183/Building-height-width/external/neurvps/data/my5_sq /users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/my5_vpts /users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/config_gsv.yaml /users/project1/pt01183/Building-height-width/external/neurvps/logs/tmm17/checkpoint_state_anetfix.pth cuda:0
[debug] backbone type: <class 'dict'>
[warn] backbone is not an nn.Module; the builder path above likely didn’t trigger. State_dict will not match until backbone is a module.
[state_dict] loaded with missing=0 unexpected=377
  unexpected (first 8): ['anet.backbone.conv1.weight', 'anet.backbone.conv1.bias', 'anet.backbone.bn1.weight', 'anet.backbone.bn1.bias', 'anet.backbone.bn1.running_mean', 'anet.backbone.bn1.running_var', 'anet.backbone.bn1.num_batches_tracked', 'anet.backbone.layer1.0